<h1>CNN-H1 3-26-2016</h1>

<strong>Abstract</strong>
Implementing the CNN-H1 using NN2 described in the paper: http://arxiv.org/pdf/1509.00244v1.pdf. 58.5% validation accuracy after training on 160 epochs.

<strong>Improvements</strong>
<ul>
<li>Take the people with the highest number of images for validation (top 1600)</li>
<li>Greyscale images - focus on the edges, features rather than colour</li>
<li>Prevent overfitting by horizontally flipping images</li>
<li>Normalize images to 160x160</li>
<li>Add Gausian noise using the following method http://stackoverflow.com/questions/33327711/how-to-add-gaussian-noise-to-an-image-using-python</li>
</ul>


<strong>Multiple GPUs experiment</strong>

Recently, it's been getting attention, hopefully keras will have some functionality for easy initializing the GPU.
<ul>
<li>It's a difficult task to get a model multiple GPUs working since it's not built in to keras</li>
<li>Both Theano and Tensorflow support multiple GPUs, but some infrastructure will have to ve created</li>
<li>At the moment, it's not worth parallelizing</li>
<li><a href="https://github.com/Theano/Theano/wiki/Using-Multiple-GPUs">Theano</a></li>
<li><a href="https://www.tensorflow.org/versions/r0.7/how_tos/using_gpu/index.html">Tensorflow</a></li>
<li><a href="http://zhqiang.org/?tag=multiple-gpu">Keras blogpost</a></li>
<li><a href="https://github.com/fchollet/keras/issues/1602">Keras issue</a>https://github.com/fchollet/keras/issues/106</li>
</ul>

In [1]:
%load_ext autoreload

In [2]:
%autoreload 2
%matplotlib inline

import os
import fnmatch
import numpy as np

from skimage import io
from skimage.color import rgb2grey
from skimage.transform import resize

from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Convolution2D, MaxPooling2D, ZeroPadding2D, AveragePooling2D
from keras.optimizers import SGD
from keras.utils import np_utils
from keras.callbacks import EarlyStopping
from keras.models import model_from_json

np.random.seed(123456)

Using Theano backend.
Using gpu device 0: GRID K520 (CNMeM is enabled with initial size: 98.0% of memory, CuDNN 3007)


In [3]:
data_path = '../data/lfw_cropped'

img_rows, img_cols = 160, 160
num_people = 600

<h2>Loading Files</h2>

In [4]:
def get_face_to_file_path_dict():
    face_to_file_paths_dict = {}
    
    for root, dirnames, filenames in os.walk(data_path):
        for dirname in dirnames:
            if dirname not in face_to_file_paths_dict:
                face_to_file_paths_dict[dirname] = []
            directory_path = os.path.join(data_path, dirname)
            for filename in os.listdir(directory_path):
                if filename.endswith(".jpg"):
                    face_to_file_paths_dict[dirname].append(os.path.join(directory_path, filename))
                            
    return face_to_file_paths_dict

In [5]:
def get_face_to_file_paths_descending_list(face_to_file_paths_dict):
    return sorted(face_to_file_paths_dict.items(), key=lambda x: len(x[1]), reverse=True)

In [6]:
face_to_file_paths_dict = get_face_to_file_path_dict()

In [7]:
face_to_file_paths_list = get_face_to_file_paths_descending_list(face_to_file_paths_dict)[:num_people]

<h2>Data Pre-Processing</h2>

In [8]:
def image_read(f):
    return resize(rgb2grey(io.imread(f)), (img_rows, img_cols))

In [9]:
def reflection(image):
    return np.array([list(reversed(row)) for row in image])

In [10]:
images_by_class = [[image_read(f) for f in x[1]] for x in face_to_file_paths_list]

In [11]:
for i in range(len(images_by_class)):
    images_by_class[i] += [reflection(image) for image in images_by_class[i]]

In [19]:
X_train = np.array([image for images in images_by_class for image in images])
y_train = np.array([images[0] for images in enumerate(images_by_class) for image in images[1]])

In [24]:
zipped = np.array(zip(X_train, y_train))
np.random.shuffle(zipped)

In [46]:
X_train = np.array([x[0] for x in zipped])
y_train = np.array([x[1] for x in zipped])

In [49]:
X_train = X_train.reshape(X_train.shape[0], 1, img_rows, img_cols)
Y_train = np_utils.to_categorical(y_train, len(images_by_class))

<h2> Training and Validation</h2>

In [50]:
def NN2(input_shape, nb_classes, nb_fc6):
    model = Sequential()
    
    # Layer 1
    model.add(Convolution2D(64, 3, 3, activation='relu', input_shape=input_shape))
    model.add(Convolution2D(128, 3, 3, activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))

    # Layer 2
    model.add(Convolution2D(64, 3, 3, activation='relu'))
    model.add(Convolution2D(128, 3, 3, activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))

    # Layer 3
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(128, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(128, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(MaxPooling2D((2,2), strides=(2,2)))

    #Layer 4
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(256, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(256, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(256, 3, 3, activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))

    #Layer 5
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(256, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(256, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(256, 3, 3))
    model.add(ZeroPadding2D((1,1)))
    model.add(AveragePooling2D((2,2), strides=(2,2)))

    model.add(Flatten())
    model.add(Dropout(0.4))
    model.add(Dense(nb_fc6))
    model.add(Dense(nb_classes, activation='softmax'))
   
    return model

In [51]:
input_shape = (1, img_rows, img_cols)
nb_fc6 = 512

In [52]:
model = NN2(input_shape, num_people, nb_fc6)
model.compile(loss='categorical_crossentropy', optimizer='sgd')
model.fit(X_train, Y_train, batch_size=32, nb_epoch=10, 
        show_accuracy=True, verbose=1, shuffle=True, validation_split=.2)

Train on 10684 samples, validate on 2672 samples
Epoch 1/10
10684/10684 [==============================] - 457s - loss: 5.9817 - acc: 0.0681 - val_loss: 5.8514 - val_acc: 0.0775
Epoch 2/10
10684/10684 [==============================] - 457s - loss: 5.7883 - acc: 0.0784 - val_loss: 5.8162 - val_acc: 0.0775
Epoch 3/10
10684/10684 [==============================] - 457s - loss: 5.7588 - acc: 0.0797 - val_loss: 5.8167 - val_acc: 0.0775
Epoch 4/10
10684/10684 [==============================] - 457s - loss: 5.7482 - acc: 0.0797 - val_loss: 5.8120 - val_acc: 0.0775
Epoch 5/10
10684/10684 [==============================] - 457s - loss: 5.7397 - acc: 0.0797 - val_loss: 5.7941 - val_acc: 0.0775
Epoch 6/10
10684/10684 [==============================] - 457s - loss: 5.7292 - acc: 0.0797 - val_loss: 5.7866 - val_acc: 0.0775
Epoch 7/10
10684/10684 [==============================] - 457s - loss: 5.6905 - acc: 0.0791 - val_loss: 5.7096 - val_acc: 0.0808
Epoch 8/10
10684/10684 [========================

In [53]:
json_string = model.to_json()
open('models/CNN-H1-10e.json', 'w').write(json_string)
model.save_weights('models/CNN-H1-10e.h5', overwrite=True)

In [54]:
model.compile(loss='categorical_crossentropy', optimizer=SGD(lr=0.001))
model.fit(X_train, Y_train, batch_size=32, nb_epoch=50,
        show_accuracy=True, verbose=1, shuffle=True, validation_split=.2)

Train on 10684 samples, validate on 2672 samples
Epoch 1/50
10684/10684 [==============================] - 457s - loss: 4.9132 - acc: 0.1330 - val_loss: 5.0781 - val_acc: 0.1317
Epoch 2/50
10684/10684 [==============================] - 457s - loss: 4.8095 - acc: 0.1462 - val_loss: 5.0272 - val_acc: 0.1407
Epoch 3/50
10684/10684 [==============================] - 457s - loss: 4.7273 - acc: 0.1547 - val_loss: 4.9269 - val_acc: 0.1632
Epoch 4/50
10684/10684 [==============================] - 457s - loss: 4.6425 - acc: 0.1694 - val_loss: 4.8435 - val_acc: 0.1725
Epoch 5/50
10684/10684 [==============================] - 457s - loss: 4.5576 - acc: 0.1823 - val_loss: 4.7709 - val_acc: 0.1778
Epoch 6/50
10684/10684 [==============================] - 457s - loss: 4.4459 - acc: 0.1971 - val_loss: 4.7010 - val_acc: 0.1815
Epoch 7/50
10684/10684 [==============================] - 457s - loss: 4.3520 - acc: 0.2105 - val_loss: 4.6055 - val_acc: 0.1984
Epoch 8/50
10684/10684 [========================

In [56]:
json_string = model.to_json()
open('models/CNN-H1-60e.json', 'w').write(json_string)
model.save_weights('models/CNN-H1-60e.h5', overwrite=True)

In [57]:
model.fit(X_train, Y_train, batch_size=32, nb_epoch=50,
        show_accuracy=True, verbose=1, shuffle=True, validation_split=.2)

Train on 10684 samples, validate on 2672 samples
Epoch 1/50
10684/10684 [==============================] - 457s - loss: 0.1101 - acc: 0.9650 - val_loss: 3.2110 - val_acc: 0.5198
Epoch 2/50
10684/10684 [==============================] - 457s - loss: 0.1156 - acc: 0.9639 - val_loss: 3.2450 - val_acc: 0.5273
Epoch 3/50
10684/10684 [==============================] - 457s - loss: 0.1168 - acc: 0.9633 - val_loss: 3.1944 - val_acc: 0.5131
Epoch 4/50
10684/10684 [==============================] - 457s - loss: 0.1122 - acc: 0.9646 - val_loss: 3.2096 - val_acc: 0.5165
Epoch 5/50
10684/10684 [==============================] - 457s - loss: 0.0989 - acc: 0.9699 - val_loss: 3.1775 - val_acc: 0.5247
Epoch 6/50
10684/10684 [==============================] - 457s - loss: 0.0903 - acc: 0.9704 - val_loss: 3.2980 - val_acc: 0.5299
Epoch 7/50
10684/10684 [==============================] - 457s - loss: 0.0878 - acc: 0.9727 - val_loss: 3.1494 - val_acc: 0.5367
Epoch 8/50
10684/10684 [========================

In [58]:
json_string = model.to_json()
open('models/CNN-H1-110e.json', 'w').write(json_string)
model.save_weights('models/CNN-H1-110e.h5', overwrite=True)

In [59]:
model.fit(X_train, Y_train, batch_size=32, nb_epoch=50,
        show_accuracy=True, verbose=1, shuffle=True, validation_split=.2)

Train on 10684 samples, validate on 2672 samples
Epoch 1/50
10684/10684 [==============================] - 457s - loss: 0.0160 - acc: 0.9946 - val_loss: 3.4093 - val_acc: 0.5565
Epoch 2/50
10684/10684 [==============================] - 457s - loss: 0.0211 - acc: 0.9934 - val_loss: 3.3031 - val_acc: 0.5689
Epoch 3/50
10684/10684 [==============================] - 457s - loss: 0.0187 - acc: 0.9942 - val_loss: 3.2700 - val_acc: 0.5662
Epoch 4/50
10684/10684 [==============================] - 457s - loss: 0.0213 - acc: 0.9934 - val_loss: 3.2296 - val_acc: 0.5677
Epoch 5/50
10684/10684 [==============================] - 457s - loss: 0.0211 - acc: 0.9942 - val_loss: 3.1410 - val_acc: 0.5786
Epoch 6/50
10684/10684 [==============================] - 457s - loss: 0.0197 - acc: 0.9943 - val_loss: 3.2682 - val_acc: 0.5737
Epoch 7/50
10684/10684 [==============================] - 457s - loss: 0.0217 - acc: 0.9935 - val_loss: 3.4874 - val_acc: 0.5606
Epoch 8/50
10684/10684 [========================

In [60]:
json_string = model.to_json()
open('models/CNN-H1-160e.json', 'w').write(json_string)
model.save_weights('models/CNN-H1-160e.h5', overwrite=True)